In [1]:
import sys

sys.path.append('..')

from src.config import SnowflakeConfig, validate_config
from src.utils.snowflake_helper import SnowflakeHelper
from snowflake.cortex import Complete
import json
from datetime import datetime

# Validate and connect
validate_config()
config = SnowflakeConfig()
sf_helper = SnowflakeHelper(config.get_connection_params())
session = sf_helper.connect()

print("✓ Staff Admin Agent initialized")

✓ Configuration validated successfully
✓ Connected to Snowflake as harismad
  Role: "DEV_ROLE"
  Warehouse: "TEST_WAREHOUSE"
  Database: "TEST_DATABASE"
  Schema: "TEST_SCHEMA"
✓ Staff Admin Agent initialized


### Define Agent Tools (Functions)

In [2]:
class StaffAdminTools:
    """Custom tools for Staff Admin Agent"""

    def __init__(self, sf_helper):
        self.sf_helper = sf_helper

    def search_sop(self, keyword: str = None, category: str = None) -> dict:
        """Search hospital SOPs by keyword or category"""
        conditions = []

        if keyword:
            conditions.append(f"(SOP_TITLE ILIKE '%{keyword}%' OR SOP_CONTENT ILIKE '%{keyword}%')")
        if category:
            conditions.append(f"SOP_CATEGORY = '{category}'")

        where_clause = " AND ".join(conditions) if conditions else "1=1"

        query = f"""
        SELECT
            SOP_ID,
            SOP_CATEGORY,
            SOP_TITLE,
            SOP_CONTENT,
            DEPARTMENT,
            LAST_UPDATED,
            VERSION
        FROM hospital_sop
        WHERE {where_clause}
        ORDER BY LAST_UPDATED DESC
        LIMIT 5
        """

        result = self.sf_helper.execute_query(query)
        return {
            "success": True,
            "count": len(result),
            "sops": result.to_dict('records')
        }

    def get_doctor_schedule(self, day: str = None, specialization: str = None,
                            doctor_name: str = None) -> dict:
        """Get doctor schedules with filters"""
        conditions = ["STATUS = 'AVAILABLE'"]

        if day:
            conditions.append(f"DAY_OF_WEEK = '{day}'")
        if specialization:
            conditions.append(f"SPECIALIZATION ILIKE '%{specialization}%'")
        if doctor_name:
            conditions.append(f"DOCTOR_NAME ILIKE '%{doctor_name}%'")

        where_clause = " AND ".join(conditions)

        query = f"""
        SELECT
            SCHEDULE_ID,
            DOCTOR_NAME,
            SPECIALIZATION,
            DAY_OF_WEEK,
            START_TIME,
            END_TIME,
            ROOM_NUMBER,
            MAX_PATIENTS,
            BOOKED_PATIENTS,
            (MAX_PATIENTS - BOOKED_PATIENTS) as AVAILABLE_SLOTS
        FROM doctor_schedule
        WHERE {where_clause}
        ORDER BY DAY_OF_WEEK, START_TIME
        LIMIT 10
        """

        result = self.sf_helper.execute_query(query)
        return {
            "success": True,
            "count": len(result),
            "schedules": result.to_dict('records')
        }

    def check_facility_availability(self, facility_type: str = None,
                                    location: str = None) -> dict:
        """Check facility availability"""
        conditions = ["STATUS = 'OPERATIONAL'"]

        if facility_type:
            conditions.append(f"FACILITY_TYPE ILIKE '%{facility_type}%'")
        if location:
            conditions.append(f"LOCATION ILIKE '%{location}%'")

        where_clause = " AND ".join(conditions)

        query = f"""
        SELECT
            FACILITY_ID,
            FACILITY_NAME,
            FACILITY_TYPE,
            LOCATION,
            CAPACITY,
            CURRENT_USAGE,
            (CAPACITY - CURRENT_USAGE) as AVAILABLE_CAPACITY,
            OPERATING_HOURS,
            CONTACT_INFO,
            STATUS
        FROM hospital_facilities
        WHERE {where_clause}
        ORDER BY AVAILABLE_CAPACITY DESC
        LIMIT 10
        """

        result = self.sf_helper.execute_query(query)
        return {
            "success": True,
            "count": len(result),
            "facilities": result.to_dict('records')
        }

    def get_department_summary(self) -> dict:
        """Get summary of hospital departments and their resources"""
        query = """
                SELECT DEPARTMENT,
                       COUNT(*)          as SOP_COUNT,
                       MAX(LAST_UPDATED) as LATEST_UPDATE
                FROM hospital_sop
                GROUP BY DEPARTMENT
                ORDER BY SOP_COUNT DESC \
                """

        result = self.sf_helper.execute_query(query)
        return {
            "success": True,
            "departments": result.to_dict('records')
        }


# Initialize tools
tools = StaffAdminTools(sf_helper)
print("✓ Staff Admin Tools initialized")

✓ Staff Admin Tools initialized


### Test Individual Tools

In [3]:
# Test 1: Search SOP
print("=== Test 1: Search Patient Care SOPs ===")
sop_result = tools.search_sop(category="Patient Care")
print(f"Found {sop_result['count']} SOPs")
if sop_result['sops']:
    print(f"Example: {sop_result['sops'][0]['SOP_TITLE']}")

# Test 2: Get Doctor Schedule
print("\n=== Test 2: Get Monday Doctor Schedules ===")
schedule_result = tools.get_doctor_schedule(day="Monday")
print(f"Found {schedule_result['count']} schedules")
if schedule_result['schedules']:
    print(
        f"Example: Dr. {schedule_result['schedules'][0]['DOCTOR_NAME']} - {schedule_result['schedules'][0]['SPECIALIZATION']}")

# Test 3: Check Facility
print("\n=== Test 3: Check Operating Room Availability ===")
facility_result = tools.check_facility_availability(facility_type="Operating Room")
print(f"Found {facility_result['count']} facilities")
if facility_result['facilities']:
    print(
        f"Example: {facility_result['facilities'][0]['FACILITY_NAME']} - {facility_result['facilities'][0]['AVAILABLE_CAPACITY']} slots available")


=== Test 1: Search Patient Care SOPs ===
Found 5 SOPs
Example: Patient Discharge Process

=== Test 2: Get Monday Doctor Schedules ===
Found 2 schedules
Example: Dr. Dr. Fitri Pratama - Pediatrician

=== Test 3: Check Operating Room Availability ===
Found 2 facilities
Example: Operating Room 1 - 1 slots available


### Build Agent Prompt System

In [4]:
def build_agent_prompt(user_query: str, tool_results: dict = None) -> str:
    """Build prompt for the Staff Admin Agent"""

    system_context = """You are AURA Staff Admin Assistant, an AI agent helping hospital administrative staff.

    Your capabilities:
    1. Search and explain hospital Standard Operating Procedures (SOPs)
    2. Check doctor schedules and availability
    3. Verify facility availability and resources
    4. Provide administrative guidance

    Guidelines:
    - Be professional and concise
    - Always cite specific SOP IDs, doctor names, or facility IDs when providing information
    - If you don't have information, admit it and suggest alternatives
    - Format responses clearly with bullet points when listing multiple items
    - Include relevant details like room numbers, timings, and contact information

    Context: You have access to hospital database with SOPs, doctor schedules, and facility information.
    Current date: {current_date}
""".format(current_date=datetime.now().strftime("%A, %B %d, %Y"))

    if tool_results:
        context_data = "\n\nAvailable Data:\n"
        for key, value in tool_results.items():
            context_data += f"\n{key}:\n{json.dumps(value, indent=2, default=str)}\n"
    else:
        context_data = ""

    full_prompt = f"""{system_context}{context_data}

User Query: {user_query}

Response:"""

    return full_prompt

### Simple Agent Implementation (Without Tool Calling)

In [5]:
def simple_agent_response(user_query: str, use_tools: bool = True) -> str:
    """
    Simple agent that uses tools based on keyword detection
    """
    query_lower = user_query.lower()
    tool_results = {}

    if use_tools:
        # Detect intent and call appropriate tools
        if any(word in query_lower for word in ['sop', 'procedure', 'protocol', 'guideline']):
            # Extract category if mentioned
            categories = ['patient care', 'emergency', 'administrative', 'safety', 'quality']
            detected_category = next((cat for cat in categories if cat in query_lower), None)

            if detected_category:
                tool_results['sop_search'] = tools.search_sop(category=detected_category.title())
            else:
                # Search by keyword
                keywords = ['admission', 'discharge', 'appointment', 'emergency', 'medication']
                detected_keyword = next((kw for kw in keywords if kw in query_lower), None)
                if detected_keyword:
                    tool_results['sop_search'] = tools.search_sop(keyword=detected_keyword)

        if any(word in query_lower for word in ['doctor', 'schedule', 'appointment', 'available']):
            # Extract day if mentioned
            days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
            detected_day = next((day for day in days if day in query_lower), None)

            # Extract specialization if mentioned
            specializations = ['cardiologist', 'pediatrician', 'orthopedist', 'dermatologist',
                               'neurologist', 'gynecologist', 'psychiatrist']
            detected_spec = next((spec for spec in specializations if spec in query_lower), None)

            if detected_day or detected_spec:
                tool_results['doctor_schedule'] = tools.get_doctor_schedule(
                    day=detected_day.title() if detected_day else None,
                    specialization=detected_spec.title() if detected_spec else None
                )

        if any(word in query_lower for word in ['facility', 'room', 'equipment', 'operating room', 'icu']):
            # Extract facility type
            facility_types = ['operating room', 'icu', 'emergency room', 'x-ray', 'mri', 'ct scanner', 'laboratory']
            detected_facility = next((ft for ft in facility_types if ft in query_lower), None)

            if detected_facility:
                tool_results['facility_availability'] = tools.check_facility_availability(
                    facility_type=detected_facility.title()
                )

    # Build prompt with tool results
    prompt = build_agent_prompt(user_query, tool_results if tool_results else None)

    # Get response from Cortex
    response = Complete(
        config.get_cortex_model(),
        prompt,
        session=session
    )

    return response

### Test Agent with Various Queries

In [6]:
test_queries = [
    "What is the SOP for patient admission?",
    "Show me doctor schedules for Monday",
    "Is there any cardiologist available this week?",
    "Check the availability of operating rooms",
    "What are the emergency procedures in case of fire?",
    "How do I schedule a patient appointment?"
]

print("=== Testing Staff Admin Agent ===\n")

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*60}")
    print(f"Query {i}: {query}")
    print(f"{'='*60}")

    response = simple_agent_response(query)
    print(f"\nAgent Response:\n{response}")
    print(f"\n{'='*60}\n")

=== Testing Staff Admin Agent ===


Query 1: What is the SOP for patient admission?


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6020\306109159.py:54: DeprecationWarning: Complete() is deprecated and will be removed in a future release. Use complete() instead
  response = Complete(



Agent Response:
 Based on the information available in our database, the Patient Admission Procedure SOP (SOP-0005) outlines the following steps:

1. Assess the situation
2. Follow department guidelines
3. Document all actions taken
4. Report to supervisor if necessary

For more detailed information, please refer to the SOP document or contact the ICU department directly.



Query 2: Show me doctor schedules for Monday


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6020\306109159.py:54: DeprecationWarning: Complete() is deprecated and will be removed in a future release. Use complete() instead
  response = Complete(



Agent Response:
 Based on the hospital database, here are the doctor schedules for Monday, October 13, 2025:

- Dr. Fitri Pratama (Pediatrician):
  * Room Number: 508
  * Start Time: 08:00:00
  * End Time: 12:00:00
  * Max Patients: 15
  * Booked Patients: 15
  * Available Slots: 0

- Dr. Ahmad Pratama (ENT Specialist):
  * Room Number: 304
  * Start Time: 09:00:00
  * End Time: 13:00:00
  * Max Patients: 15
  * Booked Patients: 6
  * Available Slots: 9

Please note that Dr. Fitri Pratama's schedule is fully booked, and Dr. Ahmad Pratama has available slots for 9 patients. If you need further assistance or information, please let me know.



Query 3: Is there any cardiologist available this week?


...


Agent Response:
 Based on the current data, I cannot directly check the doctor schedules as the provided doctor_schedule data is empty. I suggest checking with the Doctor's Scheduling department or the Hospital Information Desk for the most accurate and up-to-date information. You can contact them at extension 1234 or via email at scheduling@aura-hospital.com.

Alternatively, you can refer to the following SOP for guidelines on checking doctor availability:

SOP ID: AURA-ADM-003
Title: Checking Doctor Availability
Section: Procedure

1. Contact the Doctor's Scheduling department or the Hospital Information Desk.
2. Provide the name of the doctor and the desired date(s).
3. They will provide you with the doctor's availability for the requested date(s).



Query 4: Check the availability of operating rooms


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6020\306109159.py:54: DeprecationWarning: Complete() is deprecated and will be removed in a future release. Use complete() instead
  response = Complete(



Agent Response:
 Based on the current hospital facility availability data, here are the operating rooms and their statuses:

- Operating Room 1 (FAC-0011): Available (Current usage: 0, Available capacity: 1)
- Operating Room 2 (FAC-0019): Unavailable (Current usage: 10, Available capacity: -5)

If you would like to schedule a procedure in Operating Room 1, please contact them at ext. 2765 during their operating hours from 08:00 to 17:00. For any emergencies or urgent cases, please contact the hospital switchboard to be directed to the appropriate department.



Query 5: What are the emergency procedures in case of fire?


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6020\306109159.py:54: DeprecationWarning: Complete() is deprecated and will be removed in a future release. Use complete() instead
  response = Complete(



Agent Response:
 Based on Hospital SOP ID: HSOP-001, the following are the emergency procedures in case of a fire:

- Activate the fire alarm system.
- Evacuate the building using the nearest exit. Do not use elevators.
- Assemble at the designated assembly point outside the building.
- Call the hospital security or emergency services at extension 1234 for further assistance.
- Do not attempt to fight the fire yourself unless you are trained to do so.

Please ensure all staff and patients are aware of these procedures and practice them regularly. If you require more detailed information, please refer to the SOP document or contact the Safety and Security department.



Query 6: How do I schedule a patient appointment?

Agent Response:
 To schedule a patient appointment, kindly follow these steps:

1. Identify the specific Specialist: Determine which specialist your patient requires an appointment with. You can check doctor schedules and availability by using the following command: "Ca

### Interactive Agent Loop

In [9]:
def interactive_agent():
    """Interactive chat loop with the agent"""
    print("=== AURA Staff Admin Agent - Interactive Mode ===")
    print("Type 'exit' or 'quit' to end the conversation")
    print("Type 'help' to see available commands\n")

    conversation_history = []

    while True:
        user_input = input("\nYou: ").strip()

        if not user_input:
            continue

        if user_input.lower() in ['exit', 'quit']:
            print("Goodbye! Have a great day!")
            break

        if user_input.lower() == 'help':
            print("""
Available query types:
- SOP queries: "What is the SOP for [topic]?"
- Schedule queries: "Show doctor schedule for [day/specialization]"
- Facility queries: "Check [facility type] availability"
- General admin questions

Examples:
- "What is the patient admission procedure?"
- "Are there any cardiologists available on Monday?"
- "Show me emergency SOPs"
- "Check ICU bed availability"
            """)
            continue

        # Get agent response
        print("\nAgent: ", end="", flush=True)
        response = simple_agent_response(user_input)
        print(response)

        # Store in history
        conversation_history.append({
            "timestamp": datetime.now(),
            "user": user_input,
            "agent": response
        })

# Uncomment to run interactive mode
interactive_agent()

=== AURA Staff Admin Agent - Interactive Mode ===
Type 'exit' or 'quit' to end the conversation
Type 'help' to see available commands


Agent: 

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6020\306109159.py:54: DeprecationWarning: Complete() is deprecated and will be removed in a future release. Use complete() instead
  response = Complete(


 I am AURA Staff Admin Assistant, an AI agent designed to help hospital administrative staff with searching and explaining hospital Standard Operating Procedures (SOPs), checking doctor schedules and availability, verifying facility availability and resources, and providing administrative guidance. (SOP ID: AURA-SOP-001)
Goodbye! Have a great day!
